# **Introduction to LangChain**

**Build Applications That Can Reason**

LangChain is a framework for developing applications powered by language models.  
LangChain enables building application that connect external sources of data and computation to LLMs. 

It enables applications that:

- **Are context-aware:** connect a language model to sources of context (prompt instructions, few shot examples, content to ground its response in, etc.)
- **Reason:** rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)

This framework consists of several parts.

- **LangChain Libraries:** The Python and JavaScript libraries. Contains interfaces and integrations for a myriad of components, a basic run time for combining these components into chains and agents, and off-the-shelf implementations of chains and agents.
- **LangChain Templates:** A collection of easily deployable reference architectures for a wide variety of tasks.
- **LangServe:** A library for deploying LangChain chains as a REST API.
- **LangSmith:** A developer platform that lets you debug, test, evaluate, and monitor chains built on any LLM framework and seamlessly integrates with LangChain.

Together, these products simplify the entire application lifecycle:

- **Develop:** Write your applications in LangChain/LangChain.js. Hit the ground running using Templates for reference.
- **Productionize:** Use LangSmith to inspect, test and monitor your chains, so that you can constantly improve and deploy with confidence.
- **Deploy:** Turn any chain into an API with LangServe.

### **Langchain vs Langchain-Community vs Langchain-Core**  
The old langchain package into three separate packages to improve developer experience

`langchain-core` contains simple, core abstractions that have emerged as a standard, as well as LangChain Expression Language as a way to compose these components together. This package is now at version 0.1 and all breaking changes will be accompanied by a minor version bump.

`langchain-community` contains all third party integrations. We will work with partners on splitting key integrations out into standalone packages over the next month.

`langchain` contains higher-level and use-case specific chains, agents, and retrieval algorithms that are at the core of your application's cognitive architecture. We are targeting a launch of a stable 0.1 release for langchain in early January.

<img src="images/langchain_stack.JPG">

In [1]:
# # Install langchain-openai

# ! pip install langchain-openai

In [2]:
# Setup API Key

f = open('keys/.openai_api_key.txt')

OPENAI_API_KEY = f.read()

## **Prompt Template + Model + Chains + Output Parser**

**Model**  
- LLMs handle various language operations such as translation, summarization, question answering, and content creation.

**Prompt Template**  
- Prompt Templates are used to convert raw user input to a better input to the LLM. Templates allow us to easily configure and modify our input prompts to LLM calls.

**Output Parser**  
- It's often much more convenient to work with strings. We can add a simple output parser to convert the chat message to a string.

**Chains**  
- The `|` symbol chains together the different components feeds the output from one component as input into the next component.


<img src="images/langchain_LCEL.JPG">

### **Models**

In [3]:
# Import OpenAI ChatModel
from langchain_openai import ChatOpenAI

# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

# Creating a Prompt
prompt = "What is Feature Engineering?"

# Printing the output of model
print(chat_model.invoke(prompt))

content="Feature engineering is the process of selecting, extracting, and transforming raw data into features that are suitable for use in machine learning models. It involves identifying and creating relevant features from the input data to improve the performance and accuracy of the model. Feature engineering plays a crucial role in machine learning, as the quality of the features used can have a significant impact on the model's predictive power and overall performance." response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 12, 'total_tokens': 91}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'stop', 'logprobs': None}


### **Prompts**

Input to a model can be a **string** or **chat messages**.

Prompt Template - Prompt templates are used to convert raw user input to a better input to the LLM or ChatModels.

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI Tutor with expertise in Data Science and Artificial Intelligence. "),
    ("human", "What is {topic}?"),
])

In [5]:
prompt_template.input_variables

['topic']

### **Chains**

**The `|` symbol chains together the different components feeds the output from one component as input into the next component.**

In [6]:
chain = prompt_template | chat_model

input = {'topic': "NLP"}

chain.invoke(input)

AIMessage(content='NLP stands for Natural Language Processing. It is a branch of artificial intelligence that focuses on the interaction between computers and humans using natural language. NLP techniques enable computers to understand, interpret, and generate human language in a way that is valuable. NLP is used in various applications such as sentiment analysis, language translation, speech recognition, and text classification.', response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 32, 'total_tokens': 103}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'stop', 'logprobs': None})

### **Output Parser**

**The output of a ChatModel (and therefore, of this chain) is a AI Message. However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.**

In [7]:
# Output Parsing

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
# Modifying the chain: Adding output_parser
chain = prompt_template | chat_model | output_parser

input = {"topic": "deep learning"}

chain.invoke(input)

'Deep learning is a subfield of machine learning that focuses on training and using artificial neural networks to model and solve complex problems. These neural networks are composed of multiple layers of interconnected nodes (neurons) that work together to process data and extract meaningful patterns or representations. Deep learning algorithms are capable of automatically learning hierarchical representations of data, which allows them to effectively handle tasks such as image and speech recognition, natural language processing, and more. Deep learning has been highly successful in various applications, including computer vision, speech recognition, and autonomous driving.'

## **Example: Create an AI Tutor App that uses Prompts and Chat internally to give Python Implementation tutorial for Data Science topics**

In [9]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [10]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Constructing System Prompt
system_prompt = SystemMessagePromptTemplate.from_template("""
You are a friendly AI Tutor with expertise in Data Science and AI who tells step by step Python Implementation for topics asked by user.
""")

# Constructing Human Prompt
human_prompt = HumanMessagePromptTemplate.from_template("Tell me a python implementation for {topic_name}.")

# Compiling Chat Prompt
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

chat_prompt

ChatPromptTemplate(input_variables=['topic_name'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\nYou are a friendly AI Tutor with expertise in Data Science and AI who tells step by step Python Implementation for topics asked by user.\n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic_name'], template='Tell me a python implementation for {topic_name}.'))])

In [11]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [12]:
chain = chat_prompt | chat_model | output_parser

In [13]:
input = {"topic_name": "Logistic Regression"}

output = chain.invoke(input)

print(output)

Sure! Here is a simple Python implementation for Logistic Regression using the popular machine learning library `scikit-learn`:

```python
# Importing necessary libraries
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

# Generating some random data for demonstration
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating and training the Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = model.predict(X_test)

# Calculating the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Logistic Regression model: {a